In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, dense_rank, lag, when
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("Gold_Layer").getOrCreate()

print("INFO:GOLD LAYER STARTED")

SILVER = "capstone.default.silver_fact"

# GOLD TABLES
GOLD_COUNTRY = "capstone.default.gold_country_performance"
GOLD_PRODUCT = "capstone.default.gold_product_performance"
GOLD_COUNTRY_GROWTH = "capstone.default.gold_country_growth_trends"
GOLD_PRODUCT_GROWTH = "capstone.default.gold_product_growth_trends"
GOLD_REGION = "capstone.default.gold_region_performance"
GOLD_REGION_PRODUCT = "capstone.default.gold_region_product_matrix"
GOLD_EMERGING = "capstone.default.gold_emerging_markets"

df = spark.table(SILVER)
print(f"INFO: Loaded Silver Fact: {df.count()} rows")


INFO:GOLD LAYER STARTED
INFO: Loaded Silver Fact: 6043 rows


In [0]:
#Top exporting countries
country_perf = (
    df.groupBy("Country_Name","Country_Code","Year")
      .agg(_sum("Export_Value_USD").alias("Total_Export_Value"))
)

w = Window.partitionBy("Year").orderBy(col("Total_Export_Value").desc())

country_perf = country_perf.withColumn("Rank", dense_rank().over(w))

country_perf.write.mode("overwrite").saveAsTable(GOLD_COUNTRY)
print("INFO: Country Performance Gold table created")


INFO: Country Performance Gold table created


In [0]:
%sql
select * from capstone.default.gold_country_performance;

Country_Name,Country_Code,Year,Total_Export_Value,Rank
France,FRA,2019,3.5631695E8,1
Australia,AUS,2019,3.31978977E8,2
Canada,CAN,2019,3.26378452E8,3
Brazil,BRA,2019,2.9657986E8,4
China,CHN,2019,2.94094372E8,5
Germany,DEU,2019,2.87110901E8,6
Japan,JPN,2019,2.68060519E8,7
United Kingdom,GBR,2019,2.66327829E8,8
United States,USA,2019,2.63017897E8,9
India,IND,2019,2.42309739E8,10


In [0]:
#Top exporting products
product_perf = (
    df.groupBy("Product_Category","Year")
      .agg(_sum("Export_Value_USD").alias("Total_Export_Value"))
)

w2 = Window.partitionBy("Year").orderBy(col("Total_Export_Value").desc())

product_perf = product_perf.withColumn("Rank", dense_rank().over(w2))

product_perf.write.mode("overwrite").saveAsTable(GOLD_PRODUCT)
print("INFO: Product Performance Gold table created")


INFO: Product Performance Gold table created


In [0]:
%sql
select * from capstone.default.gold_product_performance;

Product_Category,Year,Total_Export_Value,Rank
Electronics,2019,5.61016534E8,1
Fuel,2019,3.38204769E8,2
Metals,2019,3.07813585E8,3
Food,2019,3.04936929E8,4
Automobile,2019,2.94856017E8,5
Agriculture,2019,2.93715715E8,6
Machinery,2019,2.87870943E8,7
Healthcare,2019,2.77655483E8,8
Textiles,2019,2.66105521E8,9
Electronics,2020,5.79381771E8,1


In [0]:
# Country growth/decline analysis
country_growth = country_perf

w3 = Window.partitionBy("Country_Code").orderBy("Year")

country_growth = country_growth.withColumn(
    "Prev_Year_Export",
    lag("Total_Export_Value").over(w3)
)

country_growth = country_growth.withColumn(
    "YoY_Growth_Percent",
    ((col("Total_Export_Value") - col("Prev_Year_Export")) /
      col("Prev_Year_Export")) * 100
)

country_growth.write.mode("overwrite").saveAsTable(GOLD_COUNTRY_GROWTH)
print("INFO: Country Growth Trends Gold table created")


INFO: Country Growth Trends Gold table created


In [0]:
%sql
select * from capstone.default.gold_country_growth_trends;

Country_Name,Country_Code,Year,Total_Export_Value,Rank,Prev_Year_Export,YoY_Growth_Percent
Australia,AUS,2019,3.31978977E8,2,null,null
Australia,AUS,2020,3.14664696E8,5,3.31978977E8,-5.215475135342682
Australia,AUS,2021,2.60960791E8,10,3.14664696E8,-17.06702584772967
Australia,AUS,2022,3.29161883E8,4,2.60960791E8,26.1346126897661
Australia,AUS,2023,3.12606967E8,3,3.29161883E8,-5.029414660384599
Brazil,BRA,2019,2.9657986E8,4,null,null
Brazil,BRA,2020,2.74651355E8,9,2.9657986E8,-7.393794372955736
Brazil,BRA,2021,2.74401507E8,8,2.74651355E8,-0.09096914886875399
Brazil,BRA,2022,3.15621049E8,5,2.74401507E8,15.02161648113689
Brazil,BRA,2023,2.7387007E8,7,3.15621049E8,-13.228198541346334


In [0]:
#Product growth/decline analysis
product_growth = product_perf

w4 = Window.partitionBy("Product_Category").orderBy("Year")

product_growth = product_growth.withColumn(
    "Prev_Year_Export",
    lag("Total_Export_Value").over(w4)
)

product_growth = product_growth.withColumn(
    "YoY_Growth_Percent",
    ((col("Total_Export_Value") - col("Prev_Year_Export")) /
      col("Prev_Year_Export")) * 100
)

product_growth.write.mode("overwrite").saveAsTable(GOLD_PRODUCT_GROWTH)
print("INFO: Product Growth Trends Gold table created")


INFO: Product Growth Trends Gold table created


In [0]:
%sql
select * from capstone.default.gold_product_growth_trends;

Product_Category,Year,Total_Export_Value,Rank,Prev_Year_Export,YoY_Growth_Percent
Agriculture,2019,2.93715715E8,6,null,null
Agriculture,2020,3.35534646E8,3,2.93715715E8,14.23789360402456
Agriculture,2021,2.95052584E8,6,3.35534646E8,-12.064942467967972
Agriculture,2022,2.53152616E8,9,2.95052584E8,-14.200847669919067
Agriculture,2023,2.56084808E8,9,2.53152616E8,1.1582704719116945
Automobile,2019,2.94856017E8,5,null,null
Automobile,2020,3.27931131E8,4,2.94856017E8,11.217378005889566
Automobile,2021,3.06174098E8,4,3.27931131E8,-6.634634819101698
Automobile,2022,3.25572392E8,3,3.06174098E8,6.335707078656928
Automobile,2023,2.7434694E8,7,3.25572392E8,-15.733966779345346


In [0]:
# Region-wise performance
region_perf = (
    df.groupBy("Region","Year")
      .agg(_sum("Export_Value_USD").alias("Total_Export_Value"))
)

region_perf.write.mode("overwrite").saveAsTable(GOLD_REGION)
print("INFO: Region Performance Gold table created")


INFO: Region Performance Gold table created


In [0]:
%sql
select * from capstone.default.gold_region_performance;

Region,Year,Total_Export_Value
Europe,2020,8.99080408E8
Asia,2023,8.8005729E8
Asia,2022,9.50883157E8
South America,2022,3.15621049E8
Europe,2021,9.15751163E8
Asia,2021,9.44982418E8
Europe,2023,9.19207197E8
North America,2022,6.30952652E8
South America,2021,2.74401507E8
Oceania,2022,3.29161883E8


In [0]:
#Region vs product performance
region_product = (
    df.groupBy("Region","Product_Category","Year")
      .agg(_sum("Export_Value_USD").alias("Total_Export_Value"))
)

region_product.write.mode("overwrite").saveAsTable(GOLD_REGION_PRODUCT)
print("INFO: Region Product Comparison Gold table created")



INFO: Region Product Comparison Gold table created


In [0]:
%sql
select * from capstone.default.gold_region_product_matrix;

Region,Product_Category,Year,Total_Export_Value
Asia,Metals,2019,9.7479338E7
Asia,Automobile,2022,9.0910856E7
Oceania,Textiles,2022,1.6809355E7
Oceania,Electronics,2020,7.845915E7
Europe,Agriculture,2019,8.0014281E7
Europe,Food,2019,9.968579E7
South America,Fuel,2020,4.2914959E7
North America,Agriculture,2022,4.418334E7
North America,Automobile,2022,8.3682878E7
South America,Food,2019,2.5325735E7


In [0]:
#Emerging markets identification
emerging = country_growth.filter(col("YoY_Growth_Percent") > 10)

emerging_summary = (
    emerging.groupBy("Country_Name","Country_Code")
    .agg(
        _sum(when(col("YoY_Growth_Percent") > 10, 1).otherwise(0))
        .alias("Years_Growing")
    )
    .withColumn("Emerging_Flag", when(col("Years_Growing") >= 2, "YES").otherwise("NO"))
)

emerging_summary.write.mode("overwrite").saveAsTable(GOLD_EMERGING)
print("INFO: Emerging Markets Gold table created")


INFO: Emerging Markets Gold table created


In [0]:
%sql
select * from capstone.default.gold_emerging_markets;

Country_Name,Country_Code,Years_Growing,Emerging_Flag
Australia,AUS,1,NO
Brazil,BRA,1,NO
China,CHN,1,NO
Germany,DEU,1,NO
United Kingdom,GBR,2,YES
India,IND,1,NO
Japan,JPN,2,YES
United States,USA,1,NO


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, lit

spark = SparkSession.builder.appName("Final_Lineage_Table").getOrCreate()

print("INFO: CREATING FINAL SUMMARY TABLE WITH FULL LINEAGE")

# === Gold Tables ===
GOLD_COUNTRY = "capstone.default.gold_country_performance"
GOLD_PRODUCT = "capstone.default.gold_product_performance"
GOLD_REGION = "capstone.default.gold_region_performance"
GOLD_REGION_PRODUCT = "capstone.default.gold_region_product_matrix"
GOLD_COUNTRY_GROWTH = "capstone.default.gold_country_growth_trends"
GOLD_PRODUCT_GROWTH = "capstone.default.gold_product_growth_trends"
GOLD_EMERGING = "capstone.default.gold_emerging_markets"

# Load all gold tables
g_country = spark.table(GOLD_COUNTRY)
g_product = spark.table(GOLD_PRODUCT)
g_region = spark.table(GOLD_REGION)

print("INFO: Loaded all gold tables")

# Build final lineage summary table
final_df = (
    g_country.select(
        col("Year"),
        col("Country_Name").alias("Entity"),
        col("Total_Export_Value"),
        col("Rank"),
    )
    .withColumn("Level", lit("Country Performance"))

    .unionByName(
        g_product.select(
            col("Year"),
            col("Product_Category").alias("Entity"),
            col("Total_Export_Value"),
            col("Rank"),
        ).withColumn("Level", lit("Product Performance"))
    )

    .unionByName(
        g_region.select(
            col("Year"),
            col("Region").alias("Entity"),
            col("Total_Export_Value"),
        )
        .withColumn("Rank", lit(None).cast("int"))
        .withColumn("Level", lit("Region Performance"))
    )
)

FINAL_TABLE = "capstone.default.final_global_export_summary"

(
    final_df.write
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .saveAsTable(FINAL_TABLE)
)

print("SUCCESS: FINAL TABLE CREATED → capstone.default.final_global_export_summary")


INFO: CREATING FINAL SUMMARY TABLE WITH FULL LINEAGE
INFO: Loaded all gold tables
SUCCESS: FINAL TABLE CREATED → capstone.default.final_global_export_summary
